<a href="https://colab.research.google.com/github/shreya139/Circuits-in-Stocks/blob/main/DataPrepration_project_p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Required Library

In [1]:
!pip install yfinance
!pip install fredapi
!pip install yfinance pandas numpy scikit-learn
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
ext = '0.4.0-oneiric1_amd64.deb -qO'
!wget $url/libta-lib0_$ext libta.deb
!wget $url/ta-lib0-dev_$ext ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.1) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

     ━━━━━━━━

In [2]:
pip install nsepy

  Preparing metadata (setup.py) ... done
  Created wheel for nsepy: filename=nsepy-0.8-py3-none-any.whl size=36056 sha256=ccd3909dec1da54bd870eecb23a5a44337c3d0d0040950dffc62e8061bf83892
  Stored in directory: /root/.cache/pip/wheels/4c/87/cb/acaf83f625e5fc73e1fe6e2a8e97680c74cd72391850ef5a86
Successfully built nsepy


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from talib import RSI, BBANDS, MACD, EMA, STOCH, ADX
from nsepy import get_history
from datetime import date
import yfinance as yf

In [4]:
import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as pdr

In [37]:
yf.pdr_override() # <== that's all it takes :-)


#this is reference start date before EMA 200 days will fetch null
ref_startdate = date(2008,1,1)

#actual start date and end date from where the Nifty model to be built
startdate = date(2009,1,1)
enddate = date(2022,12,31)


# download dataframe
data = pdr.get_data_yahoo("^NSEI", start=ref_startdate, end=enddate,interval='1d')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-01,6136.750000,6165.350098,6109.850098,6144.350098,6144.350098,0
2008-01-02,6144.700195,6197.000000,6060.850098,6179.399902,6179.399902,0
2008-01-03,6184.250000,6230.149902,6126.399902,6178.549805,6178.549805,0
2008-01-04,6179.100098,6300.049805,6179.100098,6274.299805,6274.299805,0
2008-01-07,6271.000000,6289.799805,6193.350098,6279.100098,6279.100098,0
...,...,...,...,...,...,...
2022-12-26,17830.400391,18084.099609,17774.250000,18014.599609,18014.599609,176700
2022-12-27,18089.800781,18149.250000,17967.449219,18132.300781,18132.300781,214300
2022-12-28,18084.750000,18173.099609,18068.349609,18122.500000,18122.500000,193900


After retrieving Nifty records, many brokerage firms utilize the TALIB API to compute key technical indicators like Exponential Moving Averages (EMA), Moving Average Convergence Divergence (MACD), Relative Strength Index (RSI -14), and Bollinger Bands values. These calculated indicators serve as the foundation for generating visually informative technical charts of the Nifty index. This practice is commonplace among various brokerage firms and is instrumental in creating insightful and actionable trading charts for Nifty.

In [6]:
#calculating technical indicators such as EMA-200, EMA-100, EMA-50, EMA-21, EMA-5, MACD, RSI-14, Bollinger Bands
EMA_200 = EMA(data['Close'],200)
EMA_100 = EMA(data['Close'],100)
EMA_50 = EMA(data['Close'],50)
EMA_21 = EMA(data['Close'],21)
EMA_5 = EMA(data['Close'],5)
data['EMA-200']=EMA_200
data['EMA-100']=EMA_100
data['EMA-50']=EMA_50
data['EMA-21']=EMA_21
data['EMA-5']=EMA_5
data['MACD'], data['macdEMA'], data['macdHistory'] = MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['RSI-14']=RSI(data['Close'],timeperiod=14)

In [7]:
print(data.head(4))

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2008-01-01  6136.750000  6165.350098  6109.850098  6144.350098  6144.350098   
2008-01-02  6144.700195  6197.000000  6060.850098  6179.399902  6179.399902   
2008-01-03  6184.250000  6230.149902  6126.399902  6178.549805  6178.549805   
2008-01-04  6179.100098  6300.049805  6179.100098  6274.299805  6274.299805   

            Volume  EMA-200  EMA-100  EMA-50  EMA-21  EMA-5  MACD  macdEMA  \
Date                                                                         
2008-01-01       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   
2008-01-02       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   
2008-01-03       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   
2008-01-04       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   

            macdHistory  RSI-14  
Date                  

In [8]:
print(data.tail(4))

                    Open          High           Low         Close  \
Date                                                                 
2022-12-27  18089.800781  18149.250000  17967.449219  18132.300781   
2022-12-28  18084.750000  18173.099609  18068.349609  18122.500000   
2022-12-29  18045.699219  18229.699219  17992.800781  18191.000000   
2022-12-30  18259.099609  18265.250000  18080.300781  18105.300781   

               Adj Close  Volume       EMA-200       EMA-100        EMA-50  \
Date                                                                         
2022-12-27  18132.300781  214300  17435.654416  17840.502471  18166.168781   
2022-12-28  18122.500000  193900  17442.488700  17846.086581  18164.456280   
2022-12-29  18191.000000  281100  17449.936574  17852.916550  18165.497210   
2022-12-30  18105.300781  192000  17456.457611  17857.914257  18163.136566   

                  EMA-21         EMA-5       MACD    macdEMA  macdHistory  \
Date                             

Retrieve historical data for significant variables such as

Brent Crude Price,

USD-INR exchange rate,

London Stock Exchange values (FTSE index),

USA stock exchange prices (Dow index),
  
Hong Kong stock exchange (Hang Seng Index), and

the India Volatility Index (VIX) from www.Investing.com .




Also, collect historical data related to

Foreign Institutional Investors (FII),

specifically focusing on FII future index values and

FII option index values.

In [9]:
# Create a ticker object for USD/INR
ticker = yf.Ticker("USDINR=X")

# Get historical data for the exchange rate
exchange_rate_data = ticker.history(start=startdate, end=enddate)

# Reset the index and convert it into a column
exchange_rate_data.reset_index(inplace=True)

# Print the historical exchange rate data
exchange_rate_data[['Date','Close']].head(2)

,Date,Close
0,2009-01-01 00:00:00+00:00,48.064999
1,2009-01-02 00:00:00+00:00,48.244999


In [10]:
data.index.dtype
# exchange_rate_data['Date'].dtype

dtype('<M8[ns]')

In [15]:
# Assuming 'Date' columns have different data types, convert both to datetime64[ns] with 'UTC' timezone
data.index = pd.to_datetime(data.index, utc=True)
exchange_rate_data['Date'] = pd.to_datetime(exchange_rate_data['Date'], utc=True)

# Sort the 'exchange_rate_data' DataFrame based on 'Date' column in ascending order
exchange_rate_data = exchange_rate_data.sort_values(by=['Date'], ascending=True)

# Now, you can perform the merge
data = pd.merge_asof(data, exchange_rate_data, on='Date')

# Print the last 2 rows of the merged DataFrame
print(data.tail(2))


                          Date        Open_x        High_x         Low_x  \
3428 2022-12-29 00:00:00+00:00  18045.699219  18229.699219  17992.800781   
3429 2022-12-30 00:00:00+00:00  18259.099609  18265.250000  18080.300781   

           Close_x     Adj Close  Volume_x       EMA-200       EMA-100  \
3428  18191.000000  18191.000000    281100  17449.936574  17852.916550   
3429  18105.300781  18105.300781    192000  17456.457611  17857.914257   

            EMA-50  ...    macdEMA  macdHistory     RSI-14     Open_y  \
3428  18165.497210  ... -11.592726   -52.402849  46.996085  82.851997   
3429  18163.136566  ... -22.859721   -45.067979  44.332909  82.835098   

         High_y      Low_y    Close_y  Volume_y  Dividends  Stock Splits  
3428  82.901199  82.740997  82.851997         0        0.0           0.0  
3429  82.836403  82.554703  82.835098         0        0.0           0.0  

[2 rows x 23 columns]


In [16]:
pip install investpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 20.6 MB/s eta 0:00:00
  Created wheel for investpy: filename=investpy-1.0.8-py3-none-any.whl size=4481567 sha256=af12efcbab758ff6b6551ae12ae2ae65ca7746de4ede6832778213636c2e2192
  Stored in directory: /root/.cache/pip/wheels/db/f4/ae/980b93b0257620bc8e4afc98a854a7a746a33eb8335fd07906
Successfully built investpy


In [38]:
# Collect data for Brent Crude Price (use 'BZ=F' for Brent Crude futures)
brent_crude_data = yf.download('BZ=F', start=ref_startdate, end=enddate)

# Collect data for London Stock Exchange (FTSE index)
ftse_data = yf.download('^FTSE', start=ref_startdate, end=enddate)

# Collect data for USA stock exchange (Dow index)
dow_data = yf.download('^DJI', start=ref_startdate, end=enddate)

# Collect data for Hong Kong stock exchange (Hang Seng Index)
hang_seng_data = yf.download('^HSI', start=ref_startdate, end=enddate)

# Collect data for India Volatility Index (VIX)
vix_data = yf.download('^IndiaVIX', start=ref_startdate, end=enddate)

# Collect historical data for Foreign Institutional Investors (FII) in India
fii_data = yf.download('FII', start=ref_startdate, end=enddate)

# Calculate EMA-5 for each dataset
# def calculate_ema(data, column, period):
#     return data[column].ewm(span=period, adjust=False).mean()

# brent_crude_data['EMA-5'] = calculate_ema(brent_crude_data, 'Close', 5)
# ftse_data['EMA-5'] = calculate_ema(ftse_data, 'Close', 5)
# dow_data['EMA-5'] = calculate_ema(dow_data, 'Close', 5)
# hang_seng_data['EMA-5'] = calculate_ema(hang_seng_data, 'Close', 5)
# vix_data['EMA-5'] = calculate_ema(vix_data, 'Close', 5)
# fii_data['EMA-5'] = calculate_ema(fii_data, 'Close', 5)

# Print or use the data as needed
print("Brent Crude")
print(brent_crude_data.head())
print("Ftse data")
print(ftse_data.head())
print("Dow data")
print(dow_data.head())
print("Hang seng data")
print(hang_seng_data.head())
print("Vix data")
print(vix_data.head())
print("Fii data")
print(fii_data.head())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^INDIAVIX']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['FII']: Exception('%ticker%: No timezone found, symbol may be delisted')



Brent Crude
                 Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2008-01-02  94.300003  98.019997  94.199997  97.839996  97.839996    1149
2008-01-03  97.500000  98.470001  97.070000  97.599998  97.599998    1386
2008-01-04  97.739998  97.889999  96.209999  96.790001  96.790001    1240
2008-01-07  96.820000  97.459999  93.730003  94.389999  94.389999    1826
2008-01-08  95.440002  96.669998  94.510002  95.540001  95.540001    1082
Ftse data
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2008-01-02  6456.899902  6512.299805  6402.600098  6416.700195  6416.700195   
2008-01-03  6416.700195  6487.799805  6394.600098  6479.399902  6479.399902   
2008-01-04  6479.399902  6534.700195  6333.200195  6348.500000  6348.500000   
2008-01-07  6348.500000  6376.500000  6275.200195  6335.700195  